In [7]:
import pandas as pd
import numpy as np

analysis_df = pd.read_csv("analysis_df.csv")

In [15]:
daily = analysis_df.groupby(['Account','date']).agg({
    'Closed PnL': 'sum',
    'Size USD': 'mean',
    'Side': 'count'
}).rename(columns={'Side': 'trades'}).reset_index()

In [16]:
sentiment_daily = analysis_df[['date','classification']].drop_duplicates()

daily = daily.merge(sentiment_daily, on='date', how='left')

In [17]:
daily['profit_flag'] = (daily['Closed PnL'] > 0).astype(int)

In [18]:
daily['sentiment'] = daily['classification'].map({
    'Fear': 0,
    'Greed': 1
})

In [19]:
X = daily[['Size USD', 'trades', 'sentiment']]
y = daily['profit_flag']

### Model Training

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LogisticRegression()
model.fit(X_train, y_train)

pred = model.predict(X_test)

print("Model Accuracy:", accuracy_score(y_test, pred))

Model Accuracy: 0.6463104325699746


In [22]:
import pickle
pickle.dump(model, open("model.pkl", "wb"))

In [23]:
daily['prediction'] = model.predict(X)
daily.to_csv("daily_predictions.csv", index=False)